*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Parte 4: Fase de Consulta (Query Phase)


### Cargue la sesión anterior
Ejecute la siguiente celda para cargar las variables que usaremos en esta parte. Como repite todo lo hecho en las partes anteriores - tardará un rato en ejecutarse.

In [ ]:
from channel import Channel
from tutorial_sessions import part1, part3 

_, _, _, _, _, _, _, f_eval, f_merkle, _ = part1()
fri_polys, fri_domains, fri_layers, fri_merkles, _ = part3()

print('Éxito!')

## Decommit en una consulta

Nuestro objetivo en esta parte es generar toda la información necesaria para verificar los compromisos (commitments) de las tres partes anteriores. En esta parte escribimos dos funciones:
1. `decommit_on_fri_layers` - envía por el channel datos que muestran que cada capa FRI es consistente con las demás, cuando se muestrea en un índice especificado.
2. `decommit_on_query` - envía los datos necesarios para descomprometerse (decommiting) en la traza y luego llama a `decommit_on_fri_layers`.  <br>

### Decommit on the FRI Layers (Descompromiso en las Capas FRI)
Implementa la función `decommit_on_fri_layers`. La función obtiene un índice y un channel, y envía por el channel la data relevante para verificar lo concerniente a las capas FRI. Específicamente, itera sobre `fri_layers` y `fri_merkles` y en cada iteración envía los siguientes datos (en el orden indicado):
1. El elemento de la capa FRI en el índice dado (usando `fri_layers`).
2. Su ruta de autenticación (utilizando el correspondiente árbol de Merkle de `fri_merkles`).
3. El hermano del elemento FRI (es decir, si el elemento es $cp_i(x)$, entonces su hermano es $cp_i(-x)$, donde $cp_i$ es el polinomio de la capa actual, y $x$ es un elemento del dominio de la capa actual). 
4. La ruta de autenticación del hermano del elemento (utilizando el mismo árbol de Merkle).

Para obtener la ruta de autenticación de un elemento, use `get_authentication_path()` de la clase `MerkleTree`, con el índice que corresponda cada vez. Tenga en cuenta que el índice del elemento hermano es igual a (idx + $\frac k  2$) mod $k$,  donde $k$ es la longitud de la capa FRI correspondiente.<br>
Nótese que **no** enviamos la ruta de autenticación del elemento en la última capa. En la última capa, todos los elementos son iguales, independientemente de la consulta, ya que son evaluaciones de un polinomio constante.

*(Recuerde convertir las variables que no sean cadenas en cadenas antes de enviarlas por el channel.)*

In [ ]:
# Arregla esto.
def decommit_on_fri_layers(idx, channel):
    for layer, merkle in zip(fri_layers[:-1], fri_merkles[:-1]):
        # Corrige esto: envía elementos y rutas de autenticación de todas las capas FRI menos la última.
        channel.send('El elemento de la capa actual')
        channel.send('La ruta de autenticación de este elemento')
        channel.send('El hermano del elemento en la capa actual')
        channel.send('La ruta de autenticación del hermano')
    # Envía el elemento de la última capa FRI.
    channel.send('El último elemento')

Solución:

In [ ]:
def decommit_on_fri_layers(idx, channel):
    for layer, merkle in zip(fri_layers[:-1], fri_merkles[:-1]):
        length = len(layer)
        idx = idx % length
        sib_idx = (idx + length // 2) % length        
        channel.send(str(layer[idx]))
        channel.send(str(merkle.get_authentication_path(idx)))
        channel.send(str(layer[sib_idx]))
        channel.send(str(merkle.get_authentication_path(sib_idx)))       
    channel.send(str(fri_layers[-1][0]))

Prueba tu código:

In [ ]:
# Prueba usando un hash previamente calculado.
test_channel = Channel()
for query in [7527, 8168, 1190, 2668, 1262, 1889, 3828, 5798, 396, 2518]:
    decommit_on_fri_layers(query, test_channel)
assert test_channel.state == 'ad4fe9aaee0fbbad0130ae0fda896393b879c5078bf57d6c705ec41ce240861b', 'El estado del channel es incorrecto.'
print('Éxito!')

### Decommit en el polinomio de traza
Para probar que efectivamente las capas FRI sobre las que nos descomprometemos se generaron a partir de la evaluación del polinomio de composición, también debemos enviar:
1. El valor $f(x)$ con su ruta de autenticación.
2. El valor $f(gx)$ con su ruta de autenticación.
3. El valor $f(g^2x)$ con su ruta de autenticación. <br>
El verificador, conociendo los coeficientes aleatorios del polinomio de composición, puede calcular su evaluación en $x$, y compararla con el primer elemento enviado de la primera capa FRI.

Por lo tanto, la función `decommit_on_query` debería enviar lo anterior (1, 2 y 3) a través del channel, y luego llamar a `decommit_on_fri_layers`. <br> <br>

Es importante destacar que, aunque $x, gx, g^2x$ son elementos consecutivos (módulo tamaño del grupo $|G|$) en la traza, las evaluaciones de `f_eval` en estos puntos son realmente 8 elementos. Esto se debe a que en la parte I "inflamos"  la traza hasta 8 veces su tamaño, para obtener un código Reed Solomon.

*Recordatorio: `f_eval` es la evaluación del polinomio de composición, y `f_merkle` es el árbol de Merkle correspondiente.*

In [ ]:
def decommit_on_query(idx, channel):
    # Envía elementos y rutas de autenticación para f(x), f(gx) y f(g^2x) por el channel. 
    channel.send('f(x)')
    channel.send('Ruta de autenticación de f(x)\'s')
    channel.send('f(gx)')
    channel.send('Ruta de autenticación de f(gx)\'s')
    channel.send('f(g^2x)')
    channel.send('Ruta de autenticación de f(g^2x)\'s')
    decommit_on_fri_layers(idx, channel) # No es necesario corregir esta línea.

Solución:

In [ ]:
def decommit_on_query(idx, channel): 
    assert idx + 16 < len(f_eval), f'índice de consulta: {idx} está fuera de rango. Longitud de la capa: {len(f_eval)}.'
    channel.send(str(f_eval[idx])) # f(x).
    channel.send(str(f_merkle.get_authentication_path(idx))) # Ruta de autenticación de f(x).
    channel.send(str(f_eval[idx + 8])) # f(gx).
    channel.send(str(f_merkle.get_authentication_path(idx + 8))) # Ruta de autenticación de f(gx).
    channel.send(str(f_eval[idx + 16])) # f(g^2x).
    channel.send(str(f_merkle.get_authentication_path(idx + 16))) # Ruta de autenticación de f(g^2x).
    decommit_on_fri_layers(idx, channel)    

Prueba tu código:

In [ ]:
# Prueba usando un hash previamente calculado.
test_channel = Channel()
for query in [8134, 1110, 1134, 6106, 7149, 4796, 144, 4738, 957]:
    decommit_on_query(query, test_channel)
assert test_channel.state == '16a72acce8d10ffb318f8f5cd557930e38cdba236a40439c9cf04aaf650cfb96', 'El estado del channel es incorrecto.'
print('Éxito!')

### Decommit on a Set of Queries (Descompromiso de un conjunto de consultas)
Para finalizar la prueba, el probador obtiene un conjunto de consultas aleatorias del channel, es decir, índices entre 0 y 8191, y se descompromete en cada consulta.

Utiliza la función que acabas de implementar `decommit_on_query()`, y `Channel.receive_random_int` para generar 3 consultas aleatorias y descomprometerse en cada una de ellas.

In [ ]:
# Arregla esto.
def decommit_fri(channel):
    for query in range(3):
        pass # Obtener un índice aleatorio del channel y enviar el descompromiso correspondiente.    

Solución:

In [ ]:
def decommit_fri(channel):
    for query in range(3):
        # Obtener un índice aleatorio del verificador y enviar el descompromiso correspondiente.
        decommit_on_query(channel.receive_random_int(0, 8191-16), channel)

Prueba tu código:

In [ ]:
test_channel = Channel()
decommit_fri(test_channel)
assert test_channel.state == 'eb96b3b77fe6cd48cfb388467c72440bdf035c51d0cfe8b4c003dd1e65e952fd', 'El estado del channel es incorrecto.' 
print('Éxito!')

## ¡Llegó la hora!
Ejecute la siguiente celda que lo une todo, ejecutando todo el código anterior, así como las funciones que escribiste en esta parte, e imprime la prueba.

In [ ]:
import time
from tutorial_sessions import part1, part3 

start = time.time()
start_all = start
print("Generando la traza...")
_, _, _, _, _, _, _, f_eval, f_merkle, _ = part1()
print(f'{time.time() - start}s')
start = time.time()
print("Generando el polinomio de composición y las capas FRI...")
fri_polys, fri_domains, fri_layers, fri_merkles, channel = part3()
print(f'{time.time() - start}s')
start = time.time()
print("Generando consultas y descompromisos...")
decommit_fri(channel)
print(f'{time.time() - start}s')
start = time.time()
print(channel.proof)
print(f'Overall time: {time.time() - start_all}s')
print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')